# Deep Learning in Medicine
### BMSC-GA 4493, BMIN-GA 3007 
### Lab 8: GAN




### Goal of this lab: 
    - Understand GAN
    - Train GAN using MNIST dataset

### Questions: 
    - What is GAN?
    - What are the different types of GANs?

In [ ]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Parameters
batch_size = 100
training_epoches = 100
lr =0.001
noise_dimension = 100

In [ ]:
#torch.manual_seed(717)
use_cuda = torch.cuda.is_available()
kwargs = {}

if use_cuda:
    #torch.cuda.manual_seed(717)
    kwargs = {'num_workers': 1, 'pin_memory': True} 

   

In [ ]:
# Data Loader
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
# Generator
class generator(nn.Module):
    # initializers
    def __init__(self, input_size=100, n_class = 28*28):
        super(generator, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, 512)
        self.fc3 = nn.Linear(self.fc2.out_features, 1024)
        self.fc4 = nn.Linear(self.fc3.out_features, n_class)

    # forward method
    def forward(self, input):
        x = F.leaky_relu(self.fc1(input), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.tanh(self.fc4(x))
        return x


In [ ]:
class discriminator(nn.Module):
    # initializers
    def __init__(self, input_size= 28*28, n_class=1):
        super(discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, 512)
        self.fc3 = nn.Linear(self.fc2.out_features, 256)
        self.fc4 = nn.Linear(self.fc3.out_features, n_class)

    # forward method
    def forward(self, input):
        x = F.leaky_relu(self.fc1(input), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.sigmoid(self.fc4(x))

        return x

In [ ]:
def show_result(num_epoch, batch_index, z, show = True, save = False, path = 'generated.png'):
    G.eval()
    test_images = G(z)
    G.train()
    
    size_figure_grid = 2
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(2, 2))
    for i in range(2):
        for j in range(1): 
            ax[i, j].get_xaxis().set_visible(False)
            ax[i, j].get_yaxis().set_visible(False)

    for k in range(4):
        i = k // 2
        j = k % 2
        ax[i, j].cla()
        ax[i, j].imshow(test_images[k, :].cpu().data.view(28, 28).numpy(), cmap='gray')

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')
    

    if show:
        plt.show()
    else:
        plt.savefig(path)
        plt.close()

In [ ]:
def train(epoch):
    loss = torch.nn.BCELoss()
    D_losses =[]
    G_losses =[]
    for batch_idx, (data, _) in enumerate(train_loader):
        # Training Discriminator 
        if batch_idx % 100 ==0:
            data = data.view(-1, 28*28)
            batch_size = data.size()[0]
            data = Variable(data)
            
            #Create the real and fake target variables and the noisy input for generator
            y_real = Variable(torch.ones(batch_size)) + (Variable(torch.randn(batch_size)) - 0.5*Variable(torch.ones(batch_size)))*0.1
            y_fake = Variable(torch.zeros(batch_size)) + (Variable(torch.randn(batch_size)) - 0.5*Variable(torch.ones(batch_size)))*0.1
            if use_cuda:
                data, y_real, y_fake = data.cuda(),  y_real.cuda(), y_fake.cuda()

            z = Variable(torch.randn(batch_size, noise_dimension))
            if use_cuda:
                z= z.cuda()
            
            # Evaluate the outputs of fake and real images using discriminator           
            # Define real loss and fake loss for the discriminator training                     
            D_result = D(data)
            D_real_loss = loss(D_result, y_real)
            G_result = G(z)
            D_result = D(G_result)
            D_fake_loss = loss(D_result, y_fake)

            D_train_loss = D_real_loss + D_fake_loss
            D_optimizer.zero_grad()
            D_train_loss.backward()
            D_optimizer.step()
            D_losses.append(D_train_loss.data[0])

        # Training generator 
        
        # Create target variables and the noisy input for generator
        z = Variable(torch.randn((batch_size, noise_dimension)))
        y = Variable(torch.ones(batch_size))
        if use_cuda:
            z, y = z.cuda(), y.cuda()
        # Evaluate the output of the fake images using discriminator     
        # Define discriminator's training loss               
        G_result = G(z)
        D_result = D(G_result)
        G_train_loss = loss(D_result, y)
        
        G_optimizer.zero_grad() 
        G_train_loss.backward()
        G_optimizer.step()
        G_losses.append(G_train_loss.data[0])
        
        if batch_idx % 30 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tDiscriminator Loss: {:.6f}\tGenerator Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),D_train_loss.data[0], G_train_loss.data[0]))
            
    z= Variable(torch.randn((4, noise_dimension)))
    if use_cuda:
        z = z.cuda()
    show_result(epoch, batch_idx, z, show = True)
    return torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))

In [ ]:
# Training Steps
G = generator(input_size = noise_dimension, n_class=28*28)
D = discriminator(input_size=28*28, n_class=1)
if use_cuda:
    G.cuda()
    D.cuda()
# Adam optimizer
G_optimizer = optim.Adam(G.parameters(), lr=lr)
D_optimizer = optim.Adam(D.parameters(), lr=lr)

D_train_losses =[]
G_train_losses =[]


for epoch in range(1, training_epoches + 1):
    D_loss, G_loss = train(epoch)
    D_train_losses.append(D_loss)
    G_train_losses.append(G_loss)

In [ ]:
# Visualize the
epoch_number = list(range(1, training_epoches + 1))
plt.plot(epoch_number,D_train_losses, 'r' )
plt.plot(epoch_number,G_train_losses, 'b' )
plt.show()

## Reference: 
    - Guide on training GANs: https://github.com/soumith/ganhacks
    - Improved techniques for training GAN: https://arxiv.org/pdf/1606.03498.pdf